In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [151]:
# Full dataset
csv_path = "/code/NetHack-Research/data/processed/new_full_data.csv"

# Load CSV to Pandas Dataframe
df = pd.read_csv(csv_path)
data = df.sort_values(['name', 'starttime'])

In [97]:
### TEMPORAL METRICS 

In [152]:
# Add game sequence number for each player
data['game_count'] = data.groupby('name').cumcount() + 1

In [153]:
# Calculate game duration in minutes
data['duration_minutes'] = (pd.to_datetime(data['endtime']) - pd.to_datetime(data['starttime'])).dt.total_seconds() / 60

In [154]:
# Calculate Inter-Session Game Time
data['next_game_time'] = data.groupby('name')['starttime'].shift(-1)
data['inter_session_time'] = (pd.to_datetime(data['next_game_time']) - pd.to_datetime(data['starttime'])).dt.total_seconds() / 3600

# Replace NaN values with a large finite value to migitate problems training 
max_value = data['inter_session_time'].max()
data['inter_session_time'] = data['inter_session_time'].fillna(max_value * 2)

data['rolling_inter_session'] = data.groupby('name')['inter_session_time'].transform(lambda x: x.rolling(3, min_periods=1).mean())

# Create quick return indicator (session within 12 hours instead of 24)
data['quick_return'] = (data['inter_session_time'] < 12).astype(int)

# Drop Next_game_time
# TODO: drop next_game_time
data = data.drop(columns='next_game_time')

In [155]:
# Ensure 'starttime' is in datetime format
data['starttime'] = pd.to_datetime(data['starttime'])

# Get first game date per player
data['first_game_date'] = data.groupby('name')['starttime'].transform('min')

# Compute days since first game
data['days_since_first_game'] = (data['starttime'] - data['first_game_date']).dt.total_seconds() / (24 * 3600)

# Ensure non-negative values (just as a safety check)
data['days_since_first_game'] = data['days_since_first_game'].clip(lower=0)

# Drop first_game_date after computation
data.drop(columns=['first_game_date'], inplace=True)

In [156]:
# Calculate days active (number of unique days played)
data['game_date'] = pd.to_datetime(data['starttime']).dt.date
temp_days = data.groupby('name')['game_date'].nunique().reset_index()
temp_days.columns = ['name', 'unique_days_active']
data = pd.merge(data, temp_days, on='name', how='left')
# TODO: Drop game_date

In [157]:
import numpy as np
import pandas as pd

# Ensure 'starttime' is in datetime format
data['starttime'] = pd.to_datetime(data['starttime'])

# Extract game date (only date, not time)
data['game_date'] = data['starttime'].dt.date

# Compute total unique days active per player
data['unique_days_active'] = data.groupby('name')['game_date'].transform('nunique')

# Compute total games per player
data['total_games'] = data.groupby('name')['name'].transform('count')

# Compute play density safely (avoid division by zero)
data['play_density'] = data['total_games'] / data['unique_days_active'].replace(0, np.nan)

# Apply log transformation to normalize the distribution
data['log_play_density'] = np.log1p(data['play_density'])

# Compute rolling play density over the last 7 played days (not just consecutive game sessions)
data['rolling_play_density'] = data.groupby('name')['game_count'].transform(
    lambda x: x.rolling(window=7, min_periods=1).sum()
)

data['smoothed_play_density'] = data.groupby('name')['rolling_play_density'].transform(lambda x: x.ewm(span=5, adjust=False).mean())

# Drop temporary columns if not needed
data.drop(columns=['total_games', 'unique_days_active', 'game_date', 'rolling_play_density', 'play_density'], inplace=True)

In [158]:
## PROGRESSION METRICS

In [159]:
# Calculate cumulative max level reached
data['cum_max_level'] = data.groupby('name')['maxlvl'].cummax()

In [160]:
# Calculate level progression rate
data['level_progression_rate'] = data['maxlvl'] / data['game_count']
# Smooth it out
data['cum_level_progression_rate'] = data.groupby('name')['level_progression_rate'].transform(lambda x: x.expanding().mean())

In [161]:
data['points_per_turn'] = data['points'] / data['turns'].where(data['turns'] > 0, np.nan)
data['log_points_per_turn'] = np.log1p(data.groupby('name')['points_per_turn'].transform(lambda x: x.rolling(5, min_periods=1).mean()))

In [162]:
# Calculate relative performance (compared to player's average)
data['avg_points'] = data.groupby('name')['points'].transform('mean')
data['relative_performance'] = data['points'] / (data['avg_points'].replace(0, np.nan) + 1e-5)

upper_limit = data['relative_performance'].quantile(0.95)
data['relative_performance'] = data['relative_performance'].clip(0, upper_limit)

In [163]:
import numpy as np
# Compute previous max level per player (for progression tracking)
data['prev_max_level'] = data.groupby('name')['cum_max_level'].shift(1).fillna(0)

# Compute level improvement per game
data['level_improvement'] = data['cum_max_level'] - data['prev_max_level']

# Ensure cumulative progression velocity starts at 0
data['cum_progression_velocity'] = data.groupby('name')['level_improvement'].cumsum()
data.loc[data['game_count'] == 1, 'cum_progression_velocity'] = 0
data['rolling_progression_velocity'] = data.groupby('name')['level_improvement'].transform(lambda x: x.rolling(5, min_periods=1).mean())
# Drop temporary column if needed
data.drop(columns=['prev_max_level'], inplace=True)

In [164]:
# Create standardized variables for composite score
persistence_components = [
    'cum_progression_velocity',
    'cum_level_progression_rate',
    'smoothed_play_density',
    'rolling_progression_velocity',
    'log_points_per_turn'
]

# Ensure all components exist
for component in persistence_components:
    if component not in data.columns:
        print(f"Warning: Component {component} not found in dataframe")
        persistence_components.remove(component)

# Standardize each component
for component in persistence_components:
    mean_val = data[component].mean()
    std_val = data[component].std()
    if std_val > 0:
        data[f'{component}_std'] = (data[component] - mean_val) / std_val
    else:
        data[f'{component}_std'] = 0

# Calculate composite persistence score
std_components = [f'{component}_std' for component in persistence_components]
data['persistence_score'] = data[std_components].mean(axis=1)

# Calculate cumulative persistence score
data['cumulative_persistence'] = data.groupby('name')['persistence_score'].cumsum()
data['avg_persistence'] = data.groupby('name')['persistence_score'].transform('mean')

print(f"Engineered {len(data.columns) - len(df.columns)} new features for persistence analysis")

Engineered 26 new features for persistence analysis


In [165]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data['game_number_centered'] = scaler.fit_transform(data[['game_count']])
data['game_number_squared'] = data['game_number_centered'] ** 2

# Calculate rolling statistics by standard deviation (e.g., 3-game windows)
window_size = 3
data['rolling_maxlvl_std'] = data.groupby('name')['maxlvl'].transform(lambda x: x.rolling(window=window_size, min_periods=1).std())
data['rolling_turns_std'] = data.groupby('name')['turns'].transform(lambda x: x.rolling(window=window_size, min_periods=1).std())

In [113]:
# Compute CV with safe division
data['rolling_maxlvl_mean'] = data.groupby('name')['maxlvl'].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
data['rolling_turns_mean'] = data.groupby('name')['turns'].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
data['rolling_maxlvl_cv'] = data['rolling_maxlvl_std'] / data['rolling_maxlvl_mean'].replace(0, np.nan).fillna(1)
data['rolling_turns_cv'] = data['rolling_turns_std'] / data['rolling_turns_mean'].replace(0, np.nan).fillna(1)

In [172]:
data.fillna(0, inplace=True)

In [176]:
scaler = StandardScaler()
data[['game_number_centered', 'game_number_squared']] = scaler.fit_transform(data[['game_number_centered', 'game_number_squared']])


In [182]:


from statsmodels.stats.outliers_influence import variance_inflation_factor
X = data[['cumulative_persistence', 'persistence_score', 'game_number_centered']]
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print(vif_data)

                  Feature       VIF
0  cumulative_persistence  2.537115
1       persistence_score  1.144682
2    game_number_centered  2.588748


In [184]:
data.to_csv("/code/NetHack-Research/data/processed/features.csv", index=False)

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

# Select relevant columns
lgm_data = data[['name', 'game_count', 'persistence_score', 'cumulative_persistence', 'game_number_centered']]

# Ensure proper data types
lgm_data = lgm_data.dropna()  # Drop missing values if needed
lgm_data = lgm_data.sort_values(['name', 'game_count'])  # Ensure time order


In [148]:
# Define LGM formula
lgm_formula = "persistence_score ~ game_number_centered + game_number_squared"

# Fit the model with random intercepts and slopes
lgm_model = smf.mixedlm(lgm_formula, lgm_data, groups=lgm_data["name"], 
                         re_formula="1 + game_number_centered")

lgm_results = lgm_model.fit()

# Print summary of model results
print(lgm_results.summary())

KeyboardInterrupt: 

In [ ]:
# Extract player-specific parameters
player_params = []
for player, effects in lgm_model.random_effects.items():
    player_params.append({
        'name': player,
        'random_intercept': effects[0],
        'random_slope': effects[1],
        'total_intercept': lgm_model.fe_params[0] + effects[0],
        'total_slope': lgm_model.fe_params[1] + effects[1]
    })

player_params_df = pd.DataFrame(player_params)

# Visualize results
plt.figure(figsize=(12, 8))
plt.scatter(
    player_params_df['total_intercept'],
    player_params_df['total_slope'],
    alpha=0.6
)
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('Initial Persistence (Intercept)')
plt.ylabel('Persistence Growth (Slope)')
plt.title(f'Player Persistence Trajectories')
plt.grid(True, alpha=0.3)

# Highlight most persistent players (high intercept, positive slope)
top_persistent = player_params_df[
    (player_params_df['total_intercept'] > player_params_df['total_intercept'].median()) &
    (player_params_df['total_slope'] > 0)
].nlargest(10, 'total_slope')

for _, player in top_persistent.iterrows():
    plt.annotate(
        player['name'],
        (player['total_intercept'], player['total_slope']),
        xytext=(5, 5),
        textcoords='offset points',
        bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
    )

plt.savefig('persistence_lgm_results.png', dpi=300, bbox_inches='tight')
plt.show()

# Save the top persistent players
print("Top persistent players identified by LGM:")
print(top_persistent[['name', 'total_intercept', 'total_slope']])
top_persistent.to_csv('top_persistent_players.csv', index=False)